# Setting openai

In [2]:
# !pip install pdfplumber

In [1]:
# !pip list

In [9]:
#pip list --format=freeze > requirements.txt

In [1]:
import os
import json
import requests
import datetime
from openai import AzureOpenAI
from dotenv import load_dotenv
import pdfplumber
import tiktoken
from datetime import datetime
from PIL import Image
import io
import re
import time

# 24.07.10 test
load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT","").strip(),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

deployment_name = os.getenv('DEPLOYMENT_NAME') # gpt-4o

# load data

In [2]:
# pdf에 있는 text list 형태로 얻기
# list index = 페이지 번호

# 자동으로 판단하기 어려움 -> flag로 책 형태 제공 (한페이지에 하나인지, 한페이지에 두 개 이상인)
def get_pdf_text(pdf_path, is_subpage):
    pdf_text_list = []

    with pdfplumber.open(pdf_path) as pdf:
        # PDF 페이지 수
        num_pages = len(pdf.pages)
        
        # 각 페이지의 텍스트 추출
        for page_num in range(num_pages):
            page = pdf.pages[page_num]
            # sub page 존재
            if is_subpage == True:
                # subpage 판단
                left = page.crop((0, 0, 0.5 * page.width, 0.9 * page.height))
                right = page.crop((0.5 * page.width, 0, page.width, page.height))
            
                l_text = left.extract_text()
                r_text = right.extract_text()

                # subpage 내용 합치기
                text = l_text + " " + r_text
            # sub page 없음
            else:
                text = page.extract_text()
            
            pdf_text_list.append(text)

    return pdf_text_list, num_pages

In [3]:
# 로미오와 줄리엣 pdf 데이터 얻기
rj_pdf_path = 'data/romeo and juliet.pdf'

rj_text_list, rj_num_pages = get_pdf_text(rj_pdf_path, is_subpage=False)
print(f'로미오와 줄리엣 페이지 수: {rj_num_pages}')

# 책 전체 토큰 수 측정
rj_str = ''.join(rj_text_list)
tokenizer = tiktoken.get_encoding("cl100k_base")
print(f'로미오와 줄리엣 토큰 수 : {len(tokenizer.encode(rj_str))}')

In [4]:
# 어린 왕자 pdf 데이터 얻기
lp_pdf_path = 'data/little_prince.pdf'

lp_text_list, lp_num_pages = get_pdf_text(lp_pdf_path, is_subpage=True)
print(f'어린왕자 페이지 수: {lp_num_pages}')

# 책 전체 토큰 수 측정
lp_str = ''.join(lp_text_list)
tokenizer = tiktoken.get_encoding("cl100k_base")
print(f'어린왕자 토큰 수 : {len(tokenizer.encode(lp_str))}')

In [39]:
# 프롬프트 정의 
prompt_dict = {
    # 핵심적인, 하나를 단어 추가
    "relation_map" : (
        "반드시 아래의 ### 읽은 페이지 내용을 기반으로 인물에 대한 관계도를 만들어. \n"
        "아래의 읽은 페이지 내용을 기반으로 핵심적인 주인공 하나를 파악하고, 반드시 파악한 주인공을 기반으로 등장하는 인물들의 관계를 표현해줘.\n"
        "아래의 json 형식으로 대답하고, 반드시 모든 등장 인물의 이름은 바꾸지 않아야 해.\n"
        "주인공과 등장 인물은 ','로 구분하여 []형태로 표현해.\n"
        "관계는 모든 인물에 대하여 인물1, 인물2, 관계로 표현해.\n\n"
        "- 주인공 : [] \n"
        "- 등장 인물 [] \n"
        "- 관계 : [인물 1, 인물 2, 관계] \n"
        "### 읽은 페이지 내용 : {context} \n\n"
        "답변을 진행한 후, 주인공의 이름을 확인하고, 모든 등장 인물들의 이름이 바뀌었는지 다시 한번 확인해줘. \n"
        "차례차례 잘 생각해보자. json 형식으로 잘 대답하면 $200의 팁을 줄게."
    ),
    
    # 그림 일기 생성 (책 제목 포함, 24.07.18 버전)
    "diary_img_with_book" : (
        "지금부터 책 제목이랑 유명한 문장을 알려줄게. 여기서 중요한 키워드를 제일 먼저 뽑아줘. \n" 
        "이 키워드를 가지고 예쁜 삽화를 만들거니까 삽화 묘사를 잘 해주는 키워드로 부탁해. 키워드는 한 줄로 정리해줘. 그림 스타일은 {img_style}야. \n"
        "-책 제목 : {book_name} \n"
        "-문장: {fav_sent} \n"
        "한 줄로 정리된 키워드를 기반으로 텍스트가 없는 동화책 삽화 이미지를 만들어줘. "
    ),
    
    # 그림 일기 생성 (책 제목 제거)
    "diary_img" : (
        "너의 유일한 역할은 그림 책에 들어가는 그림을 만드는 것이다. \n"
        "'{fav_sent}' 라는 문장을 보고 떠오르는 삽화를 생성해."
    ),

    # 줄거리 요약 + 키워드 생성
    "summary_keyword_plot" : (
        "반드시 아래의 ### 읽은 페이지 내용을 아래의 지시사항 기반으로 4개의 문장으로 요약해. \n"
        "# 지시사항 1 : ### 너가 읽은 페이지 내용을 기반으로 핵심적인 주인공 하나를 파악 \n"
        "# 지시사항 2 : 핵심적인 주인공이라 생각하고 ### 읽은 페이지 내용을 4개의 문장으로 요약 \n"
        "# 지사사항 3 : 4개의 문장 별로 아래의 형태를 가지는 json 형태로 답변 \n\n"
        "- 문장 번호 : \n"
        "- 요약 문장 : \n"
        "- 키워드 : \n\n"
        "### 읽은 페이지 내용 : {context} \n\n"
        "차례차례 잘 생각해보자. json 형식으로 잘 대답하면 $200의 팁을 줄게."
    ),

    # 줄거리 키워드에 따른 그림 생성
    "summary_img" : (
        "내가 알려주는 키워드와 이미지 스타일을 알려줄테니까 떠오르는 삽화 이미지를 만들어줘. \n"
        "- 키워드 : {keywords} \n"
        "- 이미지 스타일 : {img_style}"
    ),
    
    # KABA WIKI
    "kaba_wiki" : (
        "반드시 아래의 ### 읽은 페이지 내용을 기반으로 들어온 ### 질문에 답변해. \n\n"
        "### 질문 : {user_query} \n\n"
        "### 읽은 페이지 내용 : {context} \n\n"
        "차례차례 잘 생각해보자. json 형식으로 잘 대답하면 $200의 팁을 줄게."
    )
}

# 인물관계도

In [6]:
# 페이지 수만큼 텍스트 자르기
def slice_pdf_page(start_page, end_page, pdf_text_list):
    # 첫 idx 보정
    if start_page == 0:
        start_page = 1
    
    slice_pdf_str = ''

    try:
        for idx in range(start_page-1, end_page):
            slice_pdf_str = slice_pdf_str + '\n\n' + pdf_text_list[idx]
    except Exception as e:
        print('e:' , e)
        pass

    # slice string 보정
    mod_slice_pdf_str = [x for x in slice_pdf_str.split('\n') if len(x) > 0]
    mod_slice_pdf_str = ' '.join(mod_slice_pdf_str)

    return mod_slice_pdf_str
    
# 인물 관계도 데이터 얻기
def get_relation_map(prompt_dict, start_page, end_page, pdf_text_list):
    # 소설 시작부터 현재까지 읽은 페이지 데이터 얻기
    context_book_str = slice_pdf_page(start_page, end_page, pdf_text_list)
    
    # system에 들어갈 system 메시지 작성
    system_msg = "너는 10년동안 책 안에 있는 인물들로 인물 관계도를 만드는 전문가야. 지시사항에 맞게 인물 관계도를 만들어."

    # prompt 선언
    prompt = prompt_dict['relation_map'].format(context=context_book_str)

    # 결과
    response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role" : "system", "content" : system_msg},
            {"role" : "user", "content" : prompt}
        ]
    )

    result = response.choices[0].message.content

    # preprocessing result
    start_idx = result.find('{') # 맨 처음
    end_idx = result.rfind('}') # 맨 마지막

    relation_map_dict = json.loads(result[start_idx:end_idx+1])
    relation_map_keys = list(relation_map_dict.keys())

    main_character = relation_map_dict[relation_map_keys[0]] # 주인공
    characters = relation_map_dict[relation_map_keys[1]] # 등장인물
    relation_map = relation_map_dict[relation_map_keys[2]] # 인물관계
    # event = relation_map_dict[relation_map_keys[3]] # 사건

    return main_character, characters, relation_map, # event

In [7]:
# 어린왕자 22p까지 읽었다고 가정

book_name = '어린왕자' # 넣을지 말지 고민중, 넣으면 주인공을 어린왕자로 만듦. / 로미오와 줄리엣으로도 테스트 필요
start_page = 4
end_page = 22

main_character, characters, relation_map = get_relation_map(prompt_dict, start_page, end_page, lp_text_list) # event 제거

# (기존프롬프트) test 1 : 14.4초 / test 2 : 19.5초 / test 3 : 10.2초
# (사건내용제거) test 1 : 8.0초 / 6.4초 / 6.0초
print(f'main_character: {main_character}')
print(f'characters: {characters}')
print(f'relation_map: {relation_map}')
# print(f'event: {event}')

In [8]:
# 변환 함수
def new_list_to_json(main_character, input_list):
    nodes = {}
    
    # 모든 노드에 대해 초기화
    for parent, child, relationship in input_list:
        if parent not in nodes:
            nodes[parent] = {"name": parent, "children": []}
        if child not in nodes:
            nodes[child] = {"name": child, "relationship": relationship, "children": []}
    
    # 부모-자식 관계 설정
    for parent, child, relationship in input_list:
        child_node = nodes[child]
        child_node["relationship"] = relationship
        nodes[parent]["children"].append(child_node)
    
    # 루트 노드 생성
    root = nodes[main_character]
    return root

# JSON 변환
relation_map_json = new_list_to_json(main_character[0], relation_map)

# 아래 데이터 전달
relation_map_json

# 그림 일기 생성

- 이슈사항 1) relation map과 다른 Client를 새로 생성해야 되는지 잘 모르겠음
- 이슈사항 2) 장르(스타일?) 정보 필요?, 책 제목 프롬프트에 넣어야 하는지?
- 이슈사항 3) 그림에서 텍스트 빼는 방법 (오타가 너무 많음)
- https://community.openai.com/t/keep-dalle-from-including-text/32556/13
- https://community.openai.com/t/add-negative-prompt-important/526543/4

In [9]:
# 개발 참고
# https://www.analyticsvidhya.com/blog/2024/07/dall-e3/

In [10]:
# 이미지 생성
def generate_image(prompt, n=1, size="1024x1024"):
    try:
        response = client.images.generate(
            model="dall-e-3",
            # 원하는 화풍???? 찾기, 
            prompt=prompt,
            n=n,
            size=size,
        )
        urls = [img.url for img in response.data]
        # print(f"Generated URLs: {urls}")  # Debug print

        return urls

    except Exception as e:
        print(f"An error occurred in generate_image: {e}")

        return []

# print -> logging으로 변환?   
def save_image(url, filename):
    """
    Save an image from a URL to a file

    :param url: URL of the image
    :param filename: Name of the file to save the image
    """
    try:
        # print(f"Attempting to save image from URL: {url}")  # Debug print
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
        img = Image.open(io.BytesIO(response.content))
        img.save(filename)
        # print(f"Image saved successfully as {filename}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching the image: {e}")

    except Exception as e:
        print(f"Error saving the image: {e}")

# print -> logging으로 변환?
def save_img_by_url(gen_urls, img_file_path):
    if gen_urls:
        for i, url in enumerate(gen_urls):
            if url:  # Check if URL is not empty
                save_image(url, img_file_path)
            else:
                print(f"Empty URL for image {i+1}")
    else:
        print("No images were generated.")

In [19]:
# 좋아하는 문구 기반의 이미지 파일 저장 (책 이름 사용 플래그 추가)
def gen_diary_img(book_name, img_style, fav_sent, flag_use_book_nm, prompt_dict):
    # save path
    diary_file_path = f'img/test_diary/{book_name}_{str(flag_use_book_nm)}_diary.png'

    # 그림 일기 프롬프트 정의
    if flag_use_book_nm == False:
        # 책 이름 없이 그림 생성
        diary_prompt = prompt_dict['diary_img'].format(fav_sent=fav_sent)
    else: 
        # 책 이름 넣고 그림 생성
        diary_prompt = prompt_dict['diary_img_with_book'].format(img_style=img_style, book_name=book_name, fav_sent=fav_sent)

    # 이미지 생성
    diary_img_urls = generate_image(diary_prompt)
    
#     # 이미지 저장
#     save_start_time = time.time()
#     save_img_by_url(diary_img_urls, diary_file_path)
#     print(f'total save time: {time.time() - save_start_time:.2f} 초')
    
    return diary_img_urls

In [17]:
# # 책 이름 없이 그림 생성
# img_style = 'anime'
# book_name = '어린 왕자'
# fav_sent = '내가 좋아하는 사람이 나를 좋아해 주는 건 기적이야.'
# flag_use_book_nm = False

# # 이미지 생성 및 저장 
# gen_diary_img(book_name, fav_sent, flag_use_book_nm, prompt_dict)

# # 이미지 테스트
# Image.open(f'img/test_diary/{book_name}_{str(flag_use_book_nm)}_diary.png')

In [18]:
# 책 이름 넣고 그림 생성
img_style = 'anime'
book_name = '어린 왕자'
fav_sent = '내가 좋아하는 사람이 나를 좋아해 주는 건 기적이야.'
flag_use_book_nm = True

# 이미지 생성 및 저장 
diary_img_urls = gen_diary_img(book_name, img_style, fav_sent, flag_use_book_nm, prompt_dict)
print(diary_img_urls)

# 이미지 테스트
# Image.open(f'img/test_diary/{book_name}_{str(flag_use_book_nm)}_diary.png')

# AI 인물 채팅


- 이슈사항 1) 인물 관계도랑 이어져서 character들을 정해야 할 것 같음
- 이슈사항 2) semantic 기반의 hybrid 검색은 돈 더 많이 나가는 거 아님? -> 절약성 관련.. <br>
             "semantic_configuration": f"{search_index}-semantic-configuration" <br>
             "query_type": "vector_semantic_hybrid"
- 이슈사항 3) client 새로 선언해야 됨?

In [32]:
# rag node 생성 참조 블로그
# https://eehoeskrap.tistory.com/771

In [55]:
# search ai에 필요한 정보 선언

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT","").strip()
search_endpoint = os.getenv('AZURE_AI_SEARCH_ENDPOINT')
search_key = os.getenv("AZURE_AI_SEARCH_API_KEY")
search_index = os.getenv("AZURE_AI_SEARCH_INDEX")
embedding_model_name = "text-embedding-ada-002"

In [56]:
# system_msg로 책의 OO 역할을 부여한 다음, 유저 질문에 답변
# 24.07.17 : 말 어투 부여? 차갑게? 따뜻하게? 친절하게? 화내면서?
# 24.07.17 : 속도 줄이는 parameter? -> data source 부문?? : "strictness": 3, "top_n_documents": 5,
# 24.07.18 : stream = True -> backend에 어떻게 전달하고 다시 frontend로 어떻게 전달?
# 24.07.18 : stream = True 할 때 바로 보여야 하는 거 아닌지?

def get_ai_character_chat_fast(book_name, character, user_query):
    system_msg = f"당신의 유일한 역할은 {book_name} 책의 {character} 역할이다. {character} 역할이라고 생각하고 질문에 답변해."
    response = client.chat.completions.create(
        model=deployment_name,
        messages= [
        {
        "role" : "user",
        "content" : user_query
        }],
        max_tokens=300, # 1200
        temperature=0,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False,
        extra_body={
        "data_sources": [{
            "type": "azure_search",
            "parameters": {
                "endpoint": f"{search_endpoint}",
                "index_name": f"{search_index}",
                "semantic_configuration": f"{search_index}-semantic-configuration",
                "query_type": "vector_semantic_hybrid",
                "fields_mapping": {
                },
                "in_scope": True,
                "role_information": system_msg, # system message
                "filter": None, # 필요시 최적화?
                "strictness": 2, # default : 3 / 값을 낮추면 더 빠른 응답을 얻을 수 있지만, 정보의 정확성이 떨어질 수 있음
                "top_n_documents": 3, # default : 5 / 검색 결과 개수 설정, '3'
                "authentication": {
                "type": "api_key",
                "key": f"{search_key}"
                },
                "embedding_dependency": {
                "type": "deployment_name",
                "deployment_name": "text-embedding-ada-002"
                }
            }
            }]
        }
    )

    return response

In [57]:
%%time

# 걸리는 시간 (%%time 기준)
# 1회 : 6.7초, 2회 : 6.95초, 3회 : 6.89초, 4회 : 7.5초, 5회 : 11.3초, 6회 : 9.7초 
book_name = '어린 왕자'
character = '파일럿' 
user_query = '석양을 볼 때 무슨 감정이 들어?'

response = get_ai_character_chat_fast(book_name, character, user_query)

print(response.choices[0].message.content)

In [24]:
# 걸리는 시간 13.5초.
book_name = '로미오와 줄리엣'
character = '로미오' 
user_query = '왜 줄리엣과 도망치지 않은 거야? 너의 목숨보다 소중한 그녀잖아. 모든 것을 버려야지.'

response = get_ai_character_chat_fast(book_name, character, user_query)

print(response.choices[0].message.content)

# 지난 줄거리 요약
- 이슈사항 1) 폴라로이드 형식의 4컷 이미지가 잘 안 만들어짐 -> 지정한 4컷이 아닌 6컷, 9컷일 때 존재 (컷 순서 또한 의도한대로 되지 않는 듯 함)
- 이슈사항 2) 그림 일기 생성처럼 그림에 텍스트가 같이 적히는 경우 존재

In [47]:
# 현재까지 읽은 부분을 4개로 요약 -> 4개에서 키워드 추출
# client -> 다시 정의 필요??
def summary_plot(start_page, end_page, pdf_text_list, prompt_dict):
    # 소설 시작부터 현재까지 읽은 페이지 데이터 얻기
    context_book_str = slice_pdf_page(start_page, end_page, pdf_text_list)
    
    # keyword_system_message
    kw_sys_msg = "너의 유일한 역할은 주어진 소설 내용을 중심 사건으로 요약하는 것이다."
    
    # keyword prompt
    keyword_prompt = prompt_dict['summary_keyword_plot'].format(context=context_book_str)
    
    # 결과
    response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role" : "system", "content" : kw_sys_msg}, # system_msg
            {"role" : "user", "content" : keyword_prompt} # prompt
        ],
        # response_format={"type" : "json_object"}, # 답변 형식을 json으로 저장
    )

    result = response.choices[0].message.content
    print(f'[INFO] Finish summary plot...')
    
    return result

# preprocessing summary plot result
# json.load(result) -> list 형태 일 때 전처리
def _prep_gen_result_list_format(list_format):
    sent_list = []
    keyword_list = []

    # 겉은 list, 안에는 dict
    for dict_item in list_format:
        try:
            col_list = list(dict_item.keys())

            # 문장 번호가 없는 경우
            if len(col_list) == 2:
                sent_col_idx = 0
                keyword_col_idx = 1
            else:
                sent_col_idx = 1
                keyword_col_idx = 2

            # 요약 문장 추가
            sent = dict_item[col_list[sent_col_idx]]
            sent_list.append(sent)

            # keyword가 string이면 list로 변환
            keyword_obj = dict_item[col_list[keyword_col_idx]]

            if isinstance(keyword_obj, str):
                keyword_obj = keyword_obj.split(',')
            else:
                pass

            # bookname + '배경' + keyword
            keyword = [book_name + ' 배경'] + keyword_obj
            keyword_list.append(keyword)
        
        except Exception as e:
            print('list_format error: ', e)

    return sent_list, keyword_list

# json.load(result) -> dict 형태 일 때 전처리
def _prep_gen_result_dict_format(dict_format):
    sent_list = []
    keyword_list = []

    # 문장 key 정보
    sent_key_list = list(dict_format.keys())
    
    for sent_key in sent_key_list:
        try:
            sent_info = dict_format[sent_key]
            col_list = list(sent_info.keys())

            # 문장 번호가 없는 경우
            if len(col_list) == 2:
                sent_col_idx = 0
                keyword_col_idx = 1
            else:
                sent_col_idx = 1
                keyword_col_idx = 2

            # 요약 문장 추가
            sent = sent_info[col_list[sent_col_idx]]
            sent_list.append(sent)

            # keyword가 string이면 list로 변환
            keyword_obj = sent_info[col_list[keyword_col_idx]]

            if isinstance(keyword_obj, str):
                keyword_obj = keyword_obj.split(',')
            else:
                pass

            # bookname + '배경' + keyword
            keyword = [book_name + ' 배경'] + keyword_obj
            keyword_list.append(keyword)
        
        except Exception as e:
            print('dict format error: ', e)

    return sent_list, keyword_list

# json.load(result) -> str 형태 일 때 전처리
def _prep_gen_result_str_format(str_format):
    try:
        # 정규표현식으로 json 형태 데이터 추출
        json_objects = re.findall(r'\{\n.*?\n\}', str_format, re.DOTALL)

        # JSON 문자열을 json.loads를 통해 dict로 변환
        dict_list = [json.loads(obj) for obj in json_objects] # dict_list 안에 있는 값들은 dict 형태를 가지게 됨
    
    except Exception as e:
        print('str_format json error: ', e)
    
    sent_list = []
    keyword_list = []
    
    for dict_item in dict_list:
        try:
            col_list = list(dict_item.keys())

            # 문장 번호가 없는 경우
            if len(col_list) == 2:
                sent_col_idx = 0
                keyword_col_idx = 1
            else:
                sent_col_idx = 1
                keyword_col_idx = 2

            # 요약 문장 추가
            sent = dict_item[col_list[sent_col_idx]]
            sent_list.append(sent)

            # keyword가 string이면 list로 변환
            keyword_obj = dict_item[col_list[keyword_col_idx]]

            if isinstance(keyword_obj, str):
                keyword_obj = keyword_obj.split(',')
            else:
                pass

            # bookname + '배경' + keyword
            keyword = [book_name + ' 배경'] + keyword_obj
            keyword_list.append(keyword)

        except Exception as e:
            print('str_format error: ', e)

    return sent_list, keyword_list

# 비용 효율성을 올리기위해 최대한 json 형태를 정제하려 노력했다.
def prep_summary_result(result):
    # 재시도 한번만 시도
    attempt = 0
    while (attempt <= 1):
        try:
            print(f'[INFO] Start num {attempt} prep_summary_result...')    
            # """json 제거
            start_idx = result.find('\n')
            end_idx = result.rfind('\n')
            prep_result = result[start_idx+1:end_idx] # string

            sent_list = []
            keyword_list = []
            # string 데이터가 json으로 변환
            try:
                json_result = json.loads(prep_result) # 겉은 list, 안에는 dict
                # 겉은 list, 안에는 dict
                if isinstance(json_result, list):
                    print('[INFO] _prep_gen_result_list_format...')    
                    sent_list, keyword_list = _prep_gen_result_list_format(json_result)
                    break
                # 겉은 dict, 안에는 dict
                elif isinstance(json_result, dict):
                    print('[INFO] _prep_gen_result_dict_format...')
                    sent_list, keyword_list = _prep_gen_result_dict_format(json_result)
                    break
                else:
                    # 함수 다시 호출...
                    print('#' * 30)
                    print('Unknown error...')
                    print(json_result)
                    attempt += 1
                    
            except Exception as e2:
                # 형태 오류, 정규표현식으로 가져오기
                try:
                    print('[INFO] _prep_gen_result_str_format...')
                    ent_list, keyword_list = _prep_gen_result_str_format(prep_result)
                    break
                # 함수 다시 호출하기
                except Exception as e3:
                    print('#' * 30)
                    print('e3 error 함수 재실행: ', e3)
                    attempt += 1
        
        except Exception as e1:
            print('#' * 30)
            print('e1 error 함수 재실행: ', e1)
            attempt += 1
    
    return sent_list, keyword_list

# 지난 줄거리를 keyword로 요약한 다음 keyword 기반으로 그림 생성
def gen_summary_img(sent_keyword_list, img_style, book_name, prompt_dict):
    summary_img_url_list = []
    
    # key값들이 변할 수 있으므로 index로 접근해야 함
    for img_idx, sent_keyword in enumerate(sent_keyword_list):
        # 저장 장소 지정
        img_file_path = f'img/test_keyword/{book_name}_{img_style}_{img_idx}_.png'
        
        # init image prompt / img_style : anime, dreamscape
        summary_img_prompt = prompt_dict['summary_img'].format(img_style=img_style, keywords=', '.join(sent_keyword))

        # 이미지 생성
        summary_img_urls = generate_image(summary_img_prompt)
        summary_img_url_list.append(summary_img_urls)
        
        # 이미지 저장
        # save_img_by_url(summary_img_urls, img_file_path)
        
        print(f'[INFO] Finish generate image {img_idx}...')
        
    return summary_img_url_list
        
# 줄거리 요약 및 이미지 얻기
def get_summary_plot_img(book_name, start_page, end_page, pdf_text_list, img_style, prompt_dict):
    try:
        # 줄거리 요약
        summary_plot_result = summary_plot(start_page, end_page, pdf_text_list, prompt_dict)

        # 줄거리 요약 전처리
        sent_list, keyword_list = prep_summary_result(summary_plot_result)

        # 키워드 기반의 이미지 생성 (파일 저장)
        summary_img_url_list = gen_summary_img(keyword_list, img_style, book_name, prompt_dict)
        
        return sent_list, keyword_list, summary_img_url_list
    
    except Exception as e:
        print('get_summary_plot_img error: ', e)
        print(summary_plot_result)

In [53]:
# 줄거리 요약 및 줄거리 keyword 기반 그림 생성 (책이름, anime 기반)

book_name = '어린 왕자'
start_page = 4
end_page = 22
img_style = 'anime' # dreamscape

# 줄거리 요약 및 이미지 생성
summary_sent_list, summary_keyword_list, summary_img_url_list = get_summary_plot_img(book_name, start_page, end_page, lp_text_list, img_style, prompt_dict)

In [54]:
for idx, (summary_sent, summary_keyword, summary_img) in enumerate(zip(summary_sent_list, summary_keyword_list, summary_img_url_list)):
    print('#' * 50)
    print(f'idx: {idx}, sent: {summary_sent}')
    print(f'idx: {idx}, keyword: {summary_keyword}')
    print(f'idx: {idx}, img_url: {summary_img}')

# KABA WIKI
기능 : 모르는 문장이나 단어가 나올 때
- 1. 책 내용 기반으로 답변 -> 범위 지정이 어려움, 범위를 지정할 경우 엉뚱한 답변이 나옴...
- 2. 웹 검색을 통한 제공 -> 어디서 처리? bing-search로 개발?
- 3. 주인공이 '나' 인지? '어린 왕자' 인지??..

In [50]:
# system_msg로 책의 OO 역할을 부여한 다음, 유저 질문에 답변
def get_wiki_context_answer(book_name, start_page, end_page, book_text_list, user_query):
    # init system message
    system_msg = f"당신의 유일한 역할은 {book_name} 책의 주인공이다. 책 내용을 기반으로 질문에 답변해."
    
    # # 소설 시작부터 현재까지 읽은 페이지 데이터
    # context_book_str = slice_pdf_page(start_page, end_page, book_text_list)

    # # init prompt
    # prompt = prompt_dict['kaba_wiki'].format(context=context_book_str, user_query=user_query)
    
    response = client.chat.completions.create(
        model=deployment_name,
        messages= [
        {
        "role": "user",
        "content": user_query # prompt
        }],
        max_tokens=1200,
        temperature=0,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False,
        extra_body={
        "data_sources": [{
            "type": "azure_search",
            "parameters": {
                "endpoint": f"{search_endpoint}",
                "index_name": f"{search_index}",
                "semantic_configuration": f"{search_index}-semantic-configuration",
                "query_type": "vector_semantic_hybrid",
                "fields_mapping": {
                    # "content_fields_separator": "\n",
                    # "content_fields": None,
                    # "filepath_field": None,
                    # "title_field": "title",
                    # "url_field": None,
                    # "vector_fields": [
                    #   "text_vector"
                    # ]
                },
                "in_scope": True,
                "role_information": system_msg, # system message
                "filter": None,
                "strictness": 3,
                "top_n_documents": 5,
                "authentication": {
                "type": "api_key",
                "key": f"{search_key}"
                },
                "embedding_dependency": {
                "type": "deployment_name",
                "deployment_name": "text-embedding-ada-002"
                }
            }
            }]
        }
    )

    return response.choices[0].message.content

In [51]:
# context answer
book_name = '어린 왕자'
start_page = 4
end_page = 22

user_query = '어른들 세계에서 많이 살았다는 얘기다. 는 어떤 의미야?'
# user_query = "'나에게 양 한마리를 그려줘'는 어떤 의미야?"

response = get_wiki_context_answer(book_name, start_page, end_page, lp_text_list, user_query)
print(response)

In [52]:
# user_query
user_query = "'나에게 양 한마리를 그려줘'는 어떤 의미야?"

response = get_wiki_context_answer(book_name, start_page, end_page, lp_text_list, user_query)
print(response)